# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [8]:
import requests

In [ ]:
offset_ = 0
limit_ = 1000
# limit_ = 10
params = {'offset': offset_, 'limit': limit_, 'objStatus': 0 }
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
totalList = []
isContinue = True
while isContinue:
    res = requests.get(url, params)
    objects_data = res.json()
    params['offset'] += limit_
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    isContinue = len(objects_list) != 0
    totalList += objids

In [61]:
# totalList = totalList[:5]


[173, 851, 1259, 1284, 1285]

In [43]:
totalDetailedList = []
for item in totalList:
    res = requests.get(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{item}')
    detailedData = res.json()
    totalDetailedList.append(detailedData.get('data'))

In [65]:
import pandas as pd

# todo cycle
df = pd.json_normalize(totalDetailedList)

## Данные по первым 5 объектам

In [66]:
df.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,transportDistIndexValue,greenAreaIndexValue,airQualityIndexValue,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,173,566,63,"г Тольятти, бульвар Итальянский, д. 25",ЖК «Южный бульвар»,11,11,126,2022-06-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,851,1092,77,"д Десна, д. 6","ЖК ""Андерсен"" дом строительный номер 6",7,8,143,2022-06-30,Панель,...,1.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1259,1320,62,"г Рязань, улица Бугровка, д. 0","""СМАРТ дом на Новоселов""",27,27,440,2023-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1284,1527,77,"г Москва, район внутригородское муниципальное ...","Жилой Комплекс ""Мир Митино"" (сокращенное наиме...",27,27,1115,2024-12-31,Блоки,...,2.0,6.0,6.0,3.463,Пятницкое шоссе,Арбатско-Покровская,#0072BA,20.266667,False,[#0072BA]
4,1285,1528,77,"г Москва, район Внутригородское муниципальное ...","Жилой Комплекс ""Мир Митино"" (сокращенное наиме...",25,25,690,2022-06-30,Другое,...,3.0,6.0,6.0,3.463,Пятницкое шоссе,Арбатско-Покровская,#0072BA,21.633333,False,[#0072BA]


In [ ]:
## Save into files

df.to_excel('domrf.xlsx', index=False)
df.to_pickle("domrf.pkl")

## Create and save to Database

In [67]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)
df = df.applymap(str)
df.to_sql('houses', con=engine, if_exists='replace')


5